# Introduction
# ABC Gaming – Loyalty Points Analysis

This notebook analyzes player behavior on a real-money gaming platform using the loyalty points system defined by the company.

We will:
- Calculate playerwise loyalty points for specific time slots
- Create a full-month leaderboard
- Allocate bonus fairly among top 50 players
- Analyze the fairness of the current loyalty model

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Withdrawal Data - Withdrawal Data.csv to Withdrawal Data - Withdrawal Data.csv
Saving Deposit Data - Deposit Data.csv to Deposit Data - Deposit Data.csv
Saving Analytics Position Case Study - User Gameplay data.csv to Analytics Position Case Study - User Gameplay data.csv


In [ ]:
import pandas as pd
import numpy as np

#Data Load & Preprocessing

In [ ]:
deposit = pd.read_csv('Deposit Data - Deposit Data.csv', skiprows=3)
withdrawal = pd.read_csv('Withdrawal Data - Withdrawal Data.csv', skiprows=3)
gameplay = pd.read_csv('Analytics Position Case Study - User Gameplay data.csv', skiprows=3)

# Preview
deposit.head()

,User Id,Datetime,Amount
0,357,01-10-2022 0:03,2000
1,776,01-10-2022 0:03,2500
2,492,01-10-2022 0:06,5000
3,803,01-10-2022 0:07,5000
4,875,01-10-2022 0:09,1500


In [ ]:
withdrawal

,User Id,Datetime,Amount,Slot
0,190,2022-10-01 00:03:00,5872,S1
1,159,2022-10-01 00:16:00,9540,S1
2,164,2022-10-01 00:24:00,815,S1
3,946,2022-10-01 00:29:00,23000,S1
4,763,2022-10-01 00:40:00,9473,S1
...,...,...,...,...
3561,559,2022-10-31 23:27:00,5000,S2
3562,407,2022-10-31 23:51:00,3000,S2
3563,389,2022-10-31 23:56:00,14481,S2
3564,11,2022-10-31 23:57:00,4000,S2


In [ ]:
gameplay.head()

,User ID,Games Played,Datetime,Slot
0,851,1,2022-10-01,S1
1,717,1,2022-10-01,S1
2,456,1,2022-10-01,S1
3,424,1,2022-10-01,S1
4,845,1,2022-10-01,S1


In [ ]:
# Convert to datetime
deposit['Datetime'] = pd.to_datetime(deposit['Datetime'], dayfirst=True)
withdrawal['Datetime'] = pd.to_datetime(withdrawal['Datetime'], dayfirst=True)
gameplay['Datetime'] = pd.to_datetime(gameplay['Datetime'], dayfirst=True)

# Assign slot
def get_slot(t):
    return 'S1' if t.hour < 12 else 'S2'

deposit['Slot'] = deposit['Datetime'].apply(get_slot)
withdrawal['Slot'] = withdrawal['Datetime'].apply(get_slot)
gameplay['Slot'] = gameplay['Datetime'].apply(get_slot)

In [ ]:
# Example slot: 2nd October S1
date_filter = '2022-10-02'
slot_filter = 'S1'

dep_slot = deposit[(deposit['Datetime'].dt.date == pd.to_datetime(date_filter).date()) & (deposit['Slot'] == slot_filter)]
wd_slot = withdrawal[(withdrawal['Datetime'].dt.date == pd.to_datetime(date_filter).date()) & (withdrawal['Slot'] == slot_filter)]
gp_slot = gameplay[(gameplay['Datetime'].dt.date == pd.to_datetime(date_filter).date()) & (gameplay['Slot'] == slot_filter)]

#Loyalty Point Calculation

In [ ]:
def calculate_loyalty(dep_df, wd_df, gp_df):
    """Calculates loyalty points based on deposit, withdrawal, and gameplay data."""

    # Aggregate data
    dep_amt = dep_df.groupby('User Id')['Amount'].sum().reset_index(name='Deposit_Amount')
    dep_cnt = dep_df.groupby('User Id')['Amount'].count().reset_index(name='Deposit_Count')
    wd_amt = wd_df.groupby('User Id')['Amount'].sum().reset_index(name='Withdrawal_Amount')
    wd_cnt = wd_df.groupby('User Id')['Amount'].count().reset_index(name='Withdrawal_Count')
    gp = gp_df.groupby('User ID')['Games Played'].sum().reset_index(name='Games_Played')

    # Merge dataframes
    df = dep_amt.merge(dep_cnt, on='User Id', how='outer') \
                .merge(wd_amt, on='User Id', how='outer') \
                .merge(wd_cnt, on='User Id', how='outer') \
                .merge(gp, left_on='User Id', right_on='User ID', how='outer') \
                .fillna(0)

    df.drop(columns=['User ID'], inplace=True)

    # Calculate loyalty points
    df['Loyalty_Points'] = (
        0.01 * df['Deposit_Amount'] +
        0.005 * df['Withdrawal_Amount'] +
        0.001 * (df['Deposit_Count'] - df['Withdrawal_Count']).clip(lower=0) +
        0.2 * df['Games_Played']
    )

    return df.sort_values(by='Loyalty_Points', ascending=False)

In [ ]:
# Sort and rank
merged = merged.sort_values(by=['Loyalty_Points', 'Games_Played'], ascending=[False, False]).reset_index(drop=True)
merged['Rank'] = merged.index + 1

# Display top 10
merged.head(10)

,User Id,Deposit_Amount,Deposit_Count,Withdrawal_Amount,Withdrawal_Count,Games_Played,Loyalty_Points,Rank
0,634.0,515000.0,8.0,15737705.0,67.0,24,83843.325,1
1,99.0,1164800.0,47.0,2403141.0,15.0,10,23665.737,2
2,672.0,2158700.0,35.0,233750.0,5.0,10,22757.780,3
3,212.0,1924981.0,26.0,589850.0,4.0,1,22199.282,4
4,740.0,1738490.0,91.0,365288.0,7.0,2,19211.824,5
5,566.0,1819175.0,53.0,185071.0,3.0,183,19153.755,6
6,714.0,1676300.0,34.0,0.0,0.0,6,16764.234,7
7,421.0,878600.0,99.0,1269809.0,84.0,1557,15446.460,8
8,369.0,650000.0,13.0,1586208.0,9.0,37,14438.444,9
9,30.0,1329000.0,51.0,152145.0,1.0,13,14053.375,10


#Monthly Leaderboard

In [ ]:
# 📅 Full Month Calculation
monthly_loyalty = calculate_loyalty(deposit, withdrawal, gameplay)
monthly_loyalty = monthly_loyalty.sort_values(by=['Loyalty_Points', 'Games_Played'], ascending=[False, False]).reset_index(drop=True)
monthly_loyalty['Rank'] = monthly_loyalty.index + 1

monthly_loyalty.head(10)

,User Id,Deposit_Amount,Deposit_Count,Withdrawal_Amount,Withdrawal_Count,Games_Played,Loyalty_Points,Rank
0,634.0,515000.0,8.0,15737705.0,67.0,24,83843.325,1
1,99.0,1164800.0,47.0,2403141.0,15.0,10,23665.737,2
2,672.0,2158700.0,35.0,233750.0,5.0,10,22757.780,3
3,212.0,1924981.0,26.0,589850.0,4.0,1,22199.282,4
4,740.0,1738490.0,91.0,365288.0,7.0,2,19211.824,5
5,566.0,1819175.0,53.0,185071.0,3.0,183,19153.755,6
6,714.0,1676300.0,34.0,0.0,0.0,6,16764.234,7
7,421.0,878600.0,99.0,1269809.0,84.0,1557,15446.460,8
8,369.0,650000.0,13.0,1586208.0,9.0,37,14438.444,9
9,30.0,1329000.0,51.0,152145.0,1.0,13,14053.375,10


#KPIs

In [ ]:
# 📊 Average Metrics
avg_deposit = deposit['Amount'].mean()
avg_deposit_user = deposit.groupby('User Id')['Amount'].sum().mean()
avg_games_user = gameplay.groupby('User ID')['Games Played'].sum().mean()

print("Avg Deposit Amount:", avg_deposit)
print("Avg Deposit per User:", avg_deposit_user)
print("Avg Games per User:", avg_games_user)

Avg Deposit Amount: 5492.185399701801
Avg Deposit per User: 104669.64918032786
Avg Games per User: 355.267


#Slots

In [ ]:
# Slot: 2 Oct, S1
s1_2oct = calculate_loyalty(
    deposit[(deposit['Datetime'].dt.date == pd.to_datetime('2022-10-02').date()) & (deposit['Slot'] == 'S1')],
    withdrawal[(withdrawal['Datetime'].dt.date == pd.to_datetime('2022-10-02').date()) & (withdrawal['Slot'] == 'S1')],
    gameplay[(gameplay['Datetime'].dt.date == pd.to_datetime('2022-10-02').date()) & (gameplay['Slot'] == 'S1')]
)
s1_2oct

,User Id,Deposit_Amount,Deposit_Count,Withdrawal_Amount,Withdrawal_Count,Games_Played,Loyalty_Points
395,634.0,0.0,0.0,295671.0,1.0,0.0,1478.355
423,672.0,100000.0,1.0,60000.0,1.0,0.0,1300.000
349,566.0,35000.0,2.0,180041.0,1.0,1.0,1250.406
605,949.0,5500.0,1.0,124500.0,1.0,0.0,677.500
267,446.0,55000.0,2.0,0.0,0.0,1.0,550.202
...,...,...,...,...,...,...,...
208,0.0,0.0,0.0,0.0,0.0,1.0,0.200
26,0.0,0.0,0.0,0.0,0.0,1.0,0.200
22,0.0,0.0,0.0,0.0,0.0,1.0,0.200
33,0.0,0.0,0.0,0.0,0.0,1.0,0.200


In [ ]:
# Slot: 16 Oct, S2
s2_16oct = calculate_loyalty(
    deposit[(deposit['Datetime'].dt.date == pd.to_datetime('2022-10-16').date()) & (deposit['Slot'] == 'S2')],
    withdrawal[(withdrawal['Datetime'].dt.date == pd.to_datetime('2022-10-16').date()) & (withdrawal['Slot'] == 'S2')],
    gameplay[(gameplay['Datetime'].dt.date == pd.to_datetime('2022-10-16').date()) & (gameplay['Slot'] == 'S2')]
)
s2_16oct

,User Id,Deposit_Amount,Deposit_Count,Withdrawal_Amount,Withdrawal_Count,Games_Played,Loyalty_Points
365,634.0,0.0,0.0,298311.0,1.0,0.0,1491.555
122,212.0,99999.0,1.0,0.0,0.0,0.0,999.991
51,99.0,98000.0,2.0,0.0,0.0,0.0,980.002
15,28.0,90000.0,4.0,0.0,0.0,0.0,900.004
328,566.0,88000.0,3.0,0.0,0.0,1.0,880.203
...,...,...,...,...,...,...,...
184,0.0,0.0,0.0,0.0,0.0,1.0,0.200
19,0.0,0.0,0.0,0.0,0.0,1.0,0.200
557,0.0,0.0,0.0,0.0,0.0,1.0,0.200
555,0.0,0.0,0.0,0.0,0.0,1.0,0.200


In [ ]:
# Slot: 18 Oct, S1
s1_18oct = calculate_loyalty(
    deposit[(deposit['Datetime'].dt.date == pd.to_datetime('2022-10-18').date()) & (deposit['Slot'] == 'S1')],
    withdrawal[(withdrawal['Datetime'].dt.date == pd.to_datetime('2022-10-18').date()) & (withdrawal['Slot'] == 'S1')],
    gameplay[(gameplay['Datetime'].dt.date == pd.to_datetime('2022-10-18').date()) & (gameplay['Slot'] == 'S1')]
)
s1_18oct

,User Id,Deposit_Amount,Deposit_Count,Withdrawal_Amount,Withdrawal_Count,Games_Played,Loyalty_Points
391,634.0,0.0,0.0,544620.0,2.0,0.0,2723.100
122,208.0,170000.0,1.0,0.0,0.0,7.0,1701.401
419,673.0,90000.0,1.0,0.0,0.0,4.0,900.801
92,162.0,12000.0,1.0,130000.0,1.0,0.0,770.000
147,245.0,0.0,0.0,150000.0,1.0,0.0,750.000
...,...,...,...,...,...,...,...
590,0.0,0.0,0.0,0.0,0.0,1.0,0.200
598,0.0,0.0,0.0,0.0,0.0,1.0,0.200
600,0.0,0.0,0.0,0.0,0.0,1.0,0.200
602,0.0,0.0,0.0,0.0,0.0,1.0,0.200


In [ ]:
# Slot: 26 Oct, S2
s2_26oct = calculate_loyalty(
    deposit[(deposit['Datetime'].dt.date == pd.to_datetime('2022-10-26').date()) & (deposit['Slot'] == 'S2')],
    withdrawal[(withdrawal['Datetime'].dt.date == pd.to_datetime('2022-10-26').date()) & (withdrawal['Slot'] == 'S2')],
    gameplay[(gameplay['Datetime'].dt.date == pd.to_datetime('2022-10-26').date()) & (gameplay['Slot'] == 'S2')]
)
s2_26oct

,User Id,Deposit_Amount,Deposit_Count,Withdrawal_Amount,Withdrawal_Count,Games_Played,Loyalty_Points
447,714.0,200000.0,1.0,0.0,0.0,0.0,2000.001
227,369.0,50000.0,1.0,200343.0,1.0,1.0,1501.915
393,634.0,10000.0,1.0,227362.0,1.0,1.0,1237.010
329,538.0,120000.0,3.0,0.0,0.0,2.0,1200.403
0,2.0,90000.0,2.0,0.0,0.0,0.0,900.002
...,...,...,...,...,...,...,...
581,0.0,0.0,0.0,0.0,0.0,1.0,0.200
624,0.0,0.0,0.0,0.0,0.0,1.0,0.200
10,0.0,0.0,0.0,0.0,0.0,1.0,0.200
2,0.0,0.0,0.0,0.0,0.0,1.0,0.200


#Bonus Distribution

In [ ]:
# Top 50 Bonus Allocation
top50 = monthly_loyalty.head(50).copy()
total_loyalty = top50['Loyalty_Points'].sum()
top50['Bonus'] = top50['Loyalty_Points'] / total_loyalty * 50000

top50[['User Id', 'Loyalty_Points', 'Games_Played', 'Bonus']].head(50)

,User Id,Loyalty_Points,Games_Played,Bonus
0,634.0,83843.325,24,6638.861745
1,99.0,23665.737,10,1873.894625
2,672.0,22757.780,10,1802.000995
3,212.0,22199.282,1,1757.778142
4,740.0,19211.824,2,1521.225970
5,566.0,19153.755,183,1516.627965
6,714.0,16764.234,6,1327.421495
7,421.0,15446.460,1557,1223.077835
8,369.0,14438.444,37,1143.261357
9,30.0,14053.375,13,1112.770917


## 🎯 Bonus Distribution Logic

Bonus is distributed proportionally to loyalty points earned by the top 50 users.

**Formula used:**

Bonus = (Player Loyalty / Total Loyalty of Top 50) * ₹50,000

This rewards players who consistently engage across deposits, withdrawals, and gameplay — in line with the original intent of the loyalty system.

## ⚖️ Fairness Review – Loyalty Formula

**✅ Pros:**
- Encourages player activity and deposit engagement.
- Rewards gameplay and reduces weight of withdrawals.

**❌ Concerns:**
- Players who deposit heavily may dominate the leaderboard.
- Frequent low-budget gamers are undervalued.
- Wins/referrals are not considered.

**🔧 Suggestions:**
- Cap deposit weight per user/day to avoid pay-to-win bias.
- Add bonus for win streaks or referrals.
- Increase weight of games played or introduce activity tiers.

This would promote fair competition and reward genuine long-term user engagement.